# Crosshatching
Crosshatching is the drawing of two layers of hatching at right-angles to create a mesh-like pattern. Multiple layers in varying directions can be used to create textures. Crosshatching is often used to create tonal effects, by varying the spacing of lines or by adding additional layers of lines. Crosshatching is used in pencil drawing, but is particularly useful with pen and ink drawing, to create the impression of areas of tone, since the pen can only create a solid black line. 

Let us look at the process of creating a crosshatch drawing.</br>
![image](https://github.com/joeljose/assets/raw/master/crosshatch/land0.jpg)
</br>First we draw the edges/contours we see in our photo</br>
![image](https://github.com/joeljose/assets/raw/master/crosshatch/land1.jpg)
</br>In crosshatching, we create dark regions by drawing multiple hatches on those areas. And lighter areas contain progressively lesser number of hatches</br>
![image](https://github.com/joeljose/assets/raw/master/crosshatch/land2.jpg)</br>
![image](https://github.com/joeljose/assets/raw/master/crosshatch/land3.jpg)</br>
![image](https://github.com/joeljose/assets/raw/master/crosshatch/land3.5.jpg)</br>
![image](https://github.com/joeljose/assets/raw/master/crosshatch/land3.7.jpg)
</br>And finally we get </br>
![image](https://github.com/joeljose/assets/raw/master/crosshatch/land4.jpg)


In this project, we are trying to imitate the steps the artist did but through code, to produce crosshatching effect on portraits.

## All essential imports

In [ ]:
import os
import requests
from io import BytesIO
import tarfile
import tempfile
import cv2
from six.moves import urllib
from copy import deepcopy
from matplotlib import gridspec
from matplotlib import pyplot as plt
import numpy as np
from PIL import Image
from IPython.display import Image as IMG
%tensorflow_version 1.x
import tensorflow as tf

## Import helper methods
These methods help us perform the following tasks:
* Load the latest version of the pretrained DeepLab model
* Load the colormap from the PASCAL VOC dataset
* Adds colors to various labels, such as "pink" for people, "green" for bicycle and more
* Visualize an image, and add an overlay of colors on various regions

In [ ]:
class DeepLabModel(object):
  """Class to load deeplab model and run inference."""

  INPUT_TENSOR_NAME = 'ImageTensor:0'
  OUTPUT_TENSOR_NAME = 'SemanticPredictions:0'
  INPUT_SIZE = 513
  FROZEN_GRAPH_NAME = 'frozen_inference_graph'

  def __init__(self, tarball_path):
    """Creates and loads pretrained deeplab model."""
    self.graph = tf.Graph()

    graph_def = None
    # Extract frozen graph from tar archive.
    tar_file = tarfile.open(tarball_path)
    for tar_info in tar_file.getmembers():
      if self.FROZEN_GRAPH_NAME in os.path.basename(tar_info.name):
        file_handle = tar_file.extractfile(tar_info)
        graph_def = tf.GraphDef.FromString(file_handle.read())
        break

    tar_file.close()

    if graph_def is None:
      raise RuntimeError('Cannot find inference graph in tar archive.')

    with self.graph.as_default():
      tf.import_graph_def(graph_def, name='')

    self.sess = tf.Session(graph=self.graph)

  def run(self, image):
    """Runs inference on a single image.

    Args:
      image: A PIL.Image object, raw input image.

    Returns:
      resized_image: RGB image resized from original input image.
      seg_map: Segmentation map of `resized_image`.
    """
    width, height = image.size
    resize_ratio = 1.0 * self.INPUT_SIZE / max(width, height)
    print(width, height)
    print("Resize Ratio - {}".format(resize_ratio))
    target_size = (int(resize_ratio * width), int(resize_ratio * height))
    print(target_size)
    # target_size = (width, height)
    resized_image = image.convert('RGB').resize(target_size, Image.ANTIALIAS)
    batch_seg_map = self.sess.run(
        self.OUTPUT_TENSOR_NAME,
        feed_dict={self.INPUT_TENSOR_NAME: [np.asarray(resized_image)]})
    seg_map = batch_seg_map[0]
    return resized_image, seg_map


def create_pascal_label_colormap():
  """Creates a label colormap used in PASCAL VOC segmentation benchmark.

  Returns:
    A Colormap for visualizing segmentation results.
  """
  colormap = np.zeros((256, 3), dtype=int)
  ind = np.arange(256, dtype=int)

  for shift in reversed(range(8)):
    for channel in range(3):
      colormap[:, channel] |= ((ind >> channel) & 1) << shift
    ind >>= 3

  return colormap


def label_to_color_image(label):
  """Adds color defined by the dataset colormap to the label.

  Args:
    label: A 2D array with integer type, storing the segmentation label.

  Returns:
    result: A 2D array with floating type. The element of the array
      is the color indexed by the corresponding element in the input label
      to the PASCAL color map.

  Raises:
    ValueError: If label is not of rank 2 or its value is larger than color
      map maximum entry.
  """
  if label.ndim != 2:
    raise ValueError('Expect 2-D input label')

  colormap = create_pascal_label_colormap()

  if np.max(label) >= len(colormap):
    raise ValueError('label value too large.')

  return colormap[label]


def vis_segmentation(image, seg_map):
  """Visualizes input image, segmentation map and overlay view."""
  plt.figure(figsize=(15, 5))
  grid_spec = gridspec.GridSpec(1, 4, width_ratios=[6, 6, 6, 1])

  plt.subplot(grid_spec[0])
  plt.imshow(image)
  plt.axis('off')
  plt.title('input image')

  plt.subplot(grid_spec[1])
  seg_image = label_to_color_image(seg_map).astype(np.uint8)
  plt.imshow(seg_image)
  plt.axis('off')
  plt.title('segmentation map')

  plt.subplot(grid_spec[2])
  plt.imshow(image)
  plt.imshow(seg_image, alpha=0.7)
  plt.axis('off')
  plt.title('segmentation overlay')

  unique_labels = np.unique(seg_map)
  ax = plt.subplot(grid_spec[3])
  plt.imshow(
      FULL_COLOR_MAP[unique_labels].astype(np.uint8), interpolation='nearest')
  ax.yaxis.tick_right()
  plt.yticks(range(len(unique_labels)), LABEL_NAMES[unique_labels])
  plt.xticks([], [])
  ax.tick_params(width=0.0)
  plt.grid('off')
  plt.show()


In [ ]:
LABEL_NAMES = np.asarray([
    'background', 'aeroplane', 'bicycle', 'bird', 'boat', 'bottle', 'bus',
    'car', 'cat', 'chair', 'cow', 'diningtable', 'dog', 'horse', 'motorbike',
    'person', 'pottedplant', 'sheep', 'sofa', 'train', 'tv'
]) 

#  All category of things our model predicts

FULL_LABEL_MAP = np.arange(len(LABEL_NAMES)).reshape(len(LABEL_NAMES), 1)
FULL_COLOR_MAP = label_to_color_image(FULL_LABEL_MAP)

In [ ]:
MODEL_NAME = 'mobilenetv2_coco_voctrainaug'  # @param ['mobilenetv2_coco_voctrainaug', 'mobilenetv2_coco_voctrainval', 'xception_coco_voctrainaug', 'xception_coco_voctrainval']

_DOWNLOAD_URL_PREFIX = 'http://download.tensorflow.org/models/'
_MODEL_URLS = {
    'mobilenetv2_coco_voctrainaug':
        'deeplabv3_mnv2_pascal_train_aug_2018_01_29.tar.gz',
    'mobilenetv2_coco_voctrainval':
        'deeplabv3_mnv2_pascal_trainval_2018_01_29.tar.gz',
    'xception_coco_voctrainaug':
        'deeplabv3_pascal_train_aug_2018_01_04.tar.gz',
    'xception_coco_voctrainval':
        'deeplabv3_pascal_trainval_2018_01_04.tar.gz',
}
_TARBALL_NAME = 'deeplab_model.tar.gz'

model_dir = tempfile.mkdtemp()
tf.gfile.MakeDirs(model_dir)

download_path = os.path.join(model_dir, _TARBALL_NAME)
print('downloading model, this might take a while...')
urllib.request.urlretrieve(_DOWNLOAD_URL_PREFIX + _MODEL_URLS[MODEL_NAME],
                   download_path)
print('download completed! loading DeepLab model...')

MODEL = DeepLabModel(download_path)
print('model loaded successfully!')


In [ ]:
def run_visualization(IMAGE_NAME):
  """Inferences DeepLab model and visualizes result."""
  try:
    original_im = Image.open(IMAGE_NAME).convert('L')
  except IOError:
    print('Cannot retrieve image. Please check url: ' + url)
    return

  print('running deeplab on image')
  resized_im, seg_map = MODEL.run(original_im)
  vis_segmentation(resized_im, seg_map)
  return resized_im, seg_map

In [ ]:
def run_without_visualization(IMAGE_NAME):
  """Inferences DeepLab model and visualizes result."""
  try:
    original_im = Image.open(IMAGE_NAME).convert('L')
  except IOError:
    print('Cannot retrieve image. Please check url: ' + url)
    return

  print('running deeplab on image')
  resized_im, seg_map = MODEL.run(original_im)
  return resized_im, seg_map

## helper method to download files

In [ ]:
def download_file(url, dest_filename):
    """Downloads the file in given url"""
    if os.path.isfile(dest_filename):
        print('Already Downloaded: %s to %s' % (url, dest_filename))
        return
    print('Downloading: %s to %s' % (url, dest_filename))

    response = requests.get(url, stream=True)
    if not response.ok:
        raise Exception("Couldn't download file")

    with open(dest_filename, 'wb') as fp:
        for block in response.iter_content(1024):
            fp.write(block)

## Downloading and loading Hatch images

In [ ]:
download_file("https://github.com/joeljose/assets/raw/master/crosshatch/horizontalx.png","horizontal.png")
download_file("https://github.com/joeljose/assets/raw/master/crosshatch/leftx.png","left.png")
download_file("https://github.com/joeljose/assets/raw/master/crosshatch/rightx.png","right.png")
download_file("https://github.com/joeljose/assets/raw/master/crosshatch/vortexx.png","vortex.png")

In [ ]:
left=cv2.imread("left.png",0)
right=cv2.imread("right.png",0)
vortex=cv2.imread("vortex.png",0)
horizontal=cv2.imread("horizontal.png",0)


## Downloading face image that we want to apply crosshatch.

In [ ]:
download_file("https://github.com/joeljose/assets/raw/master/crosshatch/niiu.jpeg","face.jpg")

We apply the model to our image, and it gives back a segmentation map. We then use this segmentation map to remove the background.

In [ ]:
IMAGE_NAME = 'face.jpg'
resized_im, seg_map = run_visualization(IMAGE_NAME)

## We only need the 'person' class from our segmap

In [ ]:
LABEL_NAMES

In [ ]:
LABEL_NAMES[15]

The model creates an image called segmap which labels all pixels which are classified as "person" as 15. </br>
So let's create a new binary image that labels the background as 0(black), and person as 255(white).

In [ ]:
mapping = np.zeros([seg_map.shape[0],seg_map.shape[1]])
mapping[seg_map != 15] = 0
mapping[seg_map == 15] = 255

# Resize face image and segmentation map to our required size
I found out through trial and error that the hatch images are perfect(visually appealing) when we resize the input face image to a certain resolution. Resizing the hatch images and fitting it to the face image also works, but the hatches look ugly(in my opinion). You are welcome to try it if you want.

In [ ]:
face = cv2.imread(IMAGE_NAME,0)

height, width = face.shape

max_unit = 1200
hatch_unit = 2100
face_unit =max(width,height)
ratio=max_unit/face_unit
new_height=int(ratio*height)
new_width=int(ratio*width)
face_resized = cv2.resize(face,(new_width,new_height),Image.ANTIALIAS)
mapping_resized = cv2.resize(mapping,(new_width,new_height),Image.ANTIALIAS)
plt.imshow(mapping_resized,cmap='gray')

# Applying Layers
We use the segmentation map to make a new image which has white background and the person of interest.

In [ ]:
back_img=255*np.ones(face_resized.shape) # white image

layered_image= np.where(mapping_resized == 255, 
                         face_resized, 
                         back_img)
plt.imshow(layered_image,cmap='gray')

## Histogram 
We plot the histogram of our image.

In [ ]:
counts, bins = np.histogram(layered_image, range(257))
plt.bar(bins[:-1] - 0.5, counts, width=1, edgecolor='none')
plt.xlim([-0.5, 265.5])
plt.ylim([-0.5, 20000])
plt.show()

We need to find 3 threshold values as shown in the diagram such that the areas under the histogram are equal. We will ignore the final impulse in the histogram, since it is just the white background(value=255). 
![image](https://github.com/joeljose/assets/raw/master/crosshatch/hist.png)

In [ ]:
total=np.sum(counts[:255]) # we exclude the last white value. 


def find_thresh(val):
  cum_sum=0
  for i in range(255):
    cum_sum+=counts[i]
    if cum_sum>(val):
      return i
      
thresh1,thresh2,thresh3=find_thresh(total*0.25),find_thresh(total/2),find_thresh(total*0.75)

## Cropping out the hatch images in the dimensions of our face image

In [ ]:

left_new=left[:new_height,:new_width]
right_new=right[:new_height,:new_width]
horizontal_new=horizontal[:new_height,:new_width]
# cropping out the center of the vortex
vortex_new=vortex[((hatch_unit//2)-(new_height//2)):((hatch_unit//2)-(new_height//2)+new_height),
              ((hatch_unit//2)-(new_width//2)):((hatch_unit//2)-(new_width//2)+new_width)]

## hatch1 image

In [ ]:
hatch1 = np.where(layered_image<thresh1,right_new,back_img)
plt.imshow(hatch1, cmap="gray")


## hatch2 image

In [ ]:
hatch2 = np.where(layered_image<thresh2,left_new,back_img)
plt.imshow(hatch2, cmap="gray")

## hatch3 image

In [ ]:
hatch3 = np.where(layered_image<thresh3,horizontal_new,back_img)
#you can apply vortex effect if you want. Just comment the top one and uncomment the bottom one.
# hatch3 = np.where(layered_image<thresh3,vortex_new,back_img)   
plt.imshow(hatch3, cmap="gray")


## Now we need to blend in all three images

In [ ]:
def blend(list_images): # Blend images equally.

    equal_fraction = 1.0 / (len(list_images))

    output = np.zeros_like(list_images[0])

    for img in list_images:
        output = output + img * equal_fraction

    output = output.astype(np.uint8)
    return output

list_images = [hatch1, hatch2, hatch3]
output = blend(list_images)

plt.imshow(output,cmap="gray")


In [ ]:
cv2.imwrite("output.jpg", output)
IMG("output.jpg")

Now let us try to integrate all we have done into a single method that takes in an image and outputs the crosshatch image.

In [ ]:
def hatching(IMAGE_NAME,apply_vortex=False):

  resized_im, seg_map = run_without_visualization(IMAGE_NAME)

  mapping = np.zeros([seg_map.shape[0],seg_map.shape[1]])
  mapping[seg_map != 15] = 0
  mapping[seg_map == 15] = 255

  face = cv2.imread(IMAGE_NAME,0)
  height, width = face.shape

  max_unit = 1200
  hatch_unit = 2100

  face_unit =max(width,height)
  ratio=max_unit/face_unit
  new_height=int(ratio*height)
  new_width=int(ratio*width)

  face_resized = cv2.resize(face,(new_width,new_height),Image.ANTIALIAS)
  mapping_resized = cv2.resize(mapping,(new_width,new_height),Image.ANTIALIAS)

  back_img=255*np.ones(face_resized.shape) # white image

  layered_image= np.where(mapping_resized == 255, 
                          face_resized, 
                          back_img)
  
  counts, bins = np.histogram(layered_image, range(257))

  total=np.sum(counts[:255]) # we exclude the last white value.     
  thresh1,thresh2,thresh3=find_thresh(total*0.25),find_thresh(total/2),find_thresh(total*0.75)

  left_new = left[:new_height,:new_width]
  right_new = right[:new_height,:new_width]
  horizontal_new = horizontal[:new_height,:new_width]

  # cropping out the center of the vortex
  vortex_new=vortex[((hatch_unit//2)-(new_height//2)):((hatch_unit//2)-(new_height//2)+new_height),
                ((hatch_unit//2)-(new_width//2)):((hatch_unit//2)-(new_width//2)+new_width)]

  hatch1 = np.where(layered_image<thresh1,right_new,back_img)
  hatch2 = np.where(layered_image<thresh2,left_new,back_img)
  if apply_vortex:
    hatch3 = np.where(layered_image<thresh3,vortex_new,back_img)
  else:
    hatch3 = np.where(layered_image<thresh3,horizontal_new,back_img)

  list_images = [hatch1, hatch2, hatch3]
  output = blend(list_images)

  cv2.imwrite("new_output.jpg", output)
  

Lets try the vortex effect on a new image.
If you want to try your own image, upload it and give it as IMAGE_NAME argument to the 'hatching' method. Also skip the next code unit, where i just download a new image for trying out the vortex effect.

In [ ]:
download_file("https://github.com/joeljose/assets/raw/master/crosshatch/me.jpg","newface.jpg")

In [ ]:
hatching("newface.jpg",True)
IMG("new_output.jpg") 